In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch # deep learning
from torchvision import transforms # image preprocessing
from PIL import Image # read image
import os
import sys

/Users/chaiharsha/Documents/dog_breed_classifier/dogenv/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: dlopen(/Users/chaiharsha/Documents/dog_breed_classifier/dogenv/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN2at4_ops19empty_memory_format4callEN3c108ArrayRefIxEENS2_8optionalINS2_10ScalarTypeEEENS5_INS2_6LayoutEEENS5_INS2_6DeviceEEENS5_IbEENS5_INS2_12MemoryFormatEEE
  Referenced from: <61623A3D-DA3C-3AAD-B2F0-D363151DDB3F> /Users/chaiharsha/Documents/dog_breed_classifier/dogenv/lib/python3.10/site-packages/torchvision/image.so
  Expected in:     <66FB8649-BB87-3CD6-A177-462038DCAE02> /Users/chaiharsha/Documents/dog_breed_classifier/dogenv/lib/python3.10/site-packages/torch/lib/libtorch_cpu.dylib
  warn(f"Failed to load image Python extension: {e}")


In [2]:
# Get labels
df = pd.read_csv(os.path.join('dogbreeds/dogs.csv'))
df.head()

,filepaths,labels,data set
0,train/Afghan/001.jpg,Afghan,train
1,train/Afghan/002.jpg,Afghan,train
2,train/Afghan/003.jpg,Afghan,train
3,train/Afghan/004.jpg,Afghan,train
4,train/Afghan/005.jpg,Afghan,train


In [3]:
# define the image preprocessing transformation
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to a fixed size
    transforms.ToTensor(),          # Convert image to tensor
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])  # Normalize the image
])

In [4]:
# process images
breeds_train = []
images_train = []
breeds_test = []
images_test = []
breeds_val = []
images_val = []

for i, row in df.iterrows():
    # get label
    breed = row['labels']
    
    # get image
    img_name = f"dogbreeds/{row['filepaths']}"
    image = Image.open(img_name)
    image = transform(image)

    # put image/label in the correct list
    if row['data set'] == 'train':
        breeds_train.append(breed)
        images_train.append(image)
    elif row['data set'] == 'test':
        breeds_test.append(breed)
        images_test.append(image)
    else:
        breeds_val.append(breed)
        images_val.append(image)

    if i % 1000 == 0:
        print(f"Processed {i} images")

Processed 0 images
Processed 1000 images
Processed 2000 images
Processed 3000 images
Processed 4000 images
Processed 5000 images
Processed 6000 images
Processed 7000 images
Processed 8000 images
Processed 9000 images


In [5]:
# convert labels into indices
breed_to_idx = {breed: idx for idx, breed in enumerate(set(breeds_train))}
print(breed_to_idx)

{'Bloodhound': 0, 'Airedale': 1, 'Malinois': 2, 'Poodle': 3, 'Pit Bull': 4, 'Basset': 5, 'Groenendael': 6, 'Newfoundland': 7, 'Irish Spaniel': 8, 'Labradoodle': 9, 'American Spaniel': 10, 'Great Perenees': 11, 'Dalmation': 12, 'Rottweiler': 13, 'Cairn': 14, 'French Bulldog': 15, 'Collie': 16, 'Greyhound': 17, 'Bluetick': 18, 'Clumber': 19, 'Cockapoo': 20, 'Pekinese': 21, 'Labrador': 22, 'Great Dane': 23, 'Mex Hairless': 24, 'Cocker': 25, 'Doberman': 26, 'Rhodesian': 27, 'Siberian Husky': 28, 'Borzoi': 29, 'Pug': 30, 'Chihuahua': 31, 'German Sheperd': 32, 'Irish Wolfhound': 33, 'Schnauzer': 34, 'Shiba Inu': 35, 'Boston Terrier': 36, 'Vizsla': 37, 'Yorkie': 38, 'Bichon Frise': 39, 'Golden Retriever': 40, 'Chow': 41, 'American Hairless': 42, 'Bull Terrier': 43, 'Saint Bernard': 44, 'Bulldog': 45, 'Bull Mastiff': 46, 'Pomeranian': 47, 'Bearded Collie': 48, 'African Wild Dog': 49, 'Komondor': 50, 'Bermaise': 51, 'Beagle': 52, 'Maltese': 53, 'Basenji': 54, 'Dingo': 55, 'Scotch Terrier': 56, 

In [6]:
train_data = torch.stack(images_train)
train_labels = torch.tensor([breed_to_idx[breed] for breed in breeds_train], dtype=torch.long)
test_data = torch.stack(images_test)
test_labels = torch.tensor([breed_to_idx[breed] for breed in breeds_test], dtype=torch.long)
val_data = torch.stack(images_val)
val_labels = torch.tensor([breed_to_idx[breed] for breed in breeds_val], dtype=torch.long)

In [7]:
# double check sizes
print(train_data.shape)
print(train_labels.shape)
print(test_data.shape)
print(test_labels.shape)
print(val_data.shape)
print(val_labels.shape)

torch.Size([7946, 3, 224, 224])
torch.Size([7946])
torch.Size([700, 3, 224, 224])
torch.Size([700])
torch.Size([700, 3, 224, 224])
torch.Size([700])


In [8]:
# create datasets
train_dataset = torch.utils.data.TensorDataset(train_data, train_labels)
test_dataset = torch.utils.data.TensorDataset(test_data, test_labels)
val_dataset = torch.utils.data.TensorDataset(val_data, val_labels)

In [9]:
# create DataLoaders
batch_size = 64

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [10]:
import torch.nn as nn

# model
# model = nn.Sequential(
#     # 3x224x224
#     nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1),
#     nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1),
#     nn.ReLU(),
#     nn.MaxPool2d(kernel_size=2, stride=2),
#     nn.Dropout(0.2),
#     # 64x112x112
#     nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
#     nn.ReLU(),
#     nn.MaxPool2d(kernel_size=2, stride=2),
#     # 128x56x56
#     nn.Flatten(),
#     # 401408
#     nn.Linear(401408, 2048),
#     nn.ReLU(),
#     # 2048
#     nn.Linear(2048, 256),
#     nn.ReLU(),
#     nn.Linear(256, len(breed_to_idx))
# )
model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(150528, 8192),
    nn.ReLU(),
    nn.Linear(8192, 4096),
    nn.ReLU(),
    nn.Linear(4096, 1024),
    nn.ReLU(),
    nn.Linear(1024, 256),
    nn.ReLU(),
    nn.Linear(256, 128),
    nn.ReLU(),
    nn.Linear(128, len(breed_to_idx))
)

In [11]:
# loss function
criterion = nn.CrossEntropyLoss()

# optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [12]:
# training + validation loop
num_epochs = 12
for epoch in range(num_epochs):
    # training
    model.train()
    running_train_loss = 0.0
    
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_train_loss += loss.item()
    
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {running_train_loss/len(train_loader)}")

    # validation
    model.eval()
    running_val_loss = 0.0
    
    with torch.no_grad():
        for images, labels in val_loader:
            outputs = model(images)
            val_loss = criterion(outputs, labels)
            running_val_loss += val_loss.item()
    
    print(f"Epoch {epoch+1}/{num_epochs}, Validation Loss: {running_val_loss/len(val_loader)}")


KeyboardInterrupt: 

In [41]:
# save the model
torch.save(model.state_dict(), 'linear_model1116.pth')

In [55]:
model.load_state_dict(torch.load('model1115.pth'))

<All keys matched successfully>

In [56]:
model.eval()  
running_test_loss = 0.0
all_predictions = []
all_true_values = []

num_samples_to_display = 50
sample_count = 0  

with torch.no_grad(): 
    for batch_idx, (images, labels) in enumerate(train_loader):
        outputs = model(images)
        test_loss = criterion(outputs, labels)
        running_test_loss += test_loss.item()
        _, predicted = torch.max(outputs, 1)
        all_predictions.extend(predicted.cpu().numpy())  
        all_true_values.extend(labels.cpu().numpy()) 

        # Display predictions and labels for a few samples
        if sample_count < num_samples_to_display:
            for i in range(len(predicted)):
                if sample_count >= num_samples_to_display:
                    break
                print(f"Sample {sample_count + 1}: Predicted: {predicted[i].item()}, True Label: {labels[i].item()}")
                sample_count += 1


# Average test loss
average_test_loss = running_test_loss / len(test_loader)
print(f"Test Loss after training: {average_test_loss:.4f}")

# Compute accuracy
correct = sum(p == t for p, t in zip(all_predictions, all_true_values))
accuracy = correct / len(all_true_values) * 100
print(f"Test Accuracy: {accuracy:.2f}%")


Sample 1: Predicted: 49, True Label: 11
Sample 2: Predicted: 49, True Label: 44
Sample 3: Predicted: 49, True Label: 67
Sample 4: Predicted: 49, True Label: 30
Sample 5: Predicted: 49, True Label: 13
Sample 6: Predicted: 49, True Label: 64
Sample 7: Predicted: 49, True Label: 55
Sample 8: Predicted: 49, True Label: 22
Sample 9: Predicted: 49, True Label: 9
Sample 10: Predicted: 49, True Label: 51
Sample 11: Predicted: 49, True Label: 31
Sample 12: Predicted: 49, True Label: 60
Sample 13: Predicted: 49, True Label: 47
Sample 14: Predicted: 49, True Label: 67
Sample 15: Predicted: 49, True Label: 5
Sample 16: Predicted: 49, True Label: 45
Sample 17: Predicted: 49, True Label: 45
Sample 18: Predicted: 49, True Label: 10
Sample 19: Predicted: 49, True Label: 30
Sample 20: Predicted: 49, True Label: 32
Sample 21: Predicted: 49, True Label: 6
Sample 22: Predicted: 49, True Label: 55
Sample 23: Predicted: 49, True Label: 51
Sample 24: Predicted: 49, True Label: 50
Sample 25: Predicted: 49, Tr

KeyboardInterrupt: 